# DMV Analysis and predictions

In [38]:
import pandas as pd #general manipulation
from datetime import datetime, timedelta
import pickle #loads model
import ipywidgets as widgets #intercative model selection
import constants #local file containing information about the dataset

## Intro
#### Motivation
The DMV is an entity that provides essential services such as providing/renewing driver’s licenses and IDs.

During the 2020 pandemic the NJ DMV stopped accepting walk-ins and all visits needed to be booked online ahead of time. Since the system transitioned to be online only, frequently there are no available appointments for the following 90 days and one needs to keep trying to schedule an appointment.
The model developed for this project aims to help people to schedule their appointments by providing them a better window of time to go online and book the service they need. 

This is a personal project, one that started when I tried to renew my driver's license and couldn't because there wasn't available times.

The NJ MVC Appointment Scheduling can be found <a href="https://telegov.njportal.com/njmvc/AppointmentWizard">here</a>.

## Project structure
This project is split in three main parts: 
* data collection;
* data manipulation and model training, and
* project description and model deployment. 

The reason for this division is that having the data collection as its own file allows it to be run in parallel and that havig the model deployment segregated allows is to run without the dataset present.

## Data collection
The project starts by scrapping the DMV website using Python and Selenium to get the next appointment available for all locations for all services at a given time and storing them in a SQL database. This data collection was being executed every approximately two minutes, a balance between not stressing the website, having too many duplicate entries and data granularity. This scrip run on Raspberry Pi in parallel to any analysis and was executed for a few months.

## Manipulation steps

### Data structure
Each service has a list of all locations, even if a given location didn't offer that specific service during the data collection time window. 
Each location for each service had its very next available appointment collected every approximately two minutes, and if no available appointment existed, it was completed with a NULL.

### Extracting data structure
The sqlite_schema was used to retrieve all services, as each one is a table in the database. From the tables, all columns names were retrieved to get all location names.

### Cleaning NULLS
A SQL query was executed to replace the words NULLs with the actual NULL values.

### Retrieving the right time
The shift (am/pm) information was lost during the scrapping data, hence they needed to be reintroduced to the SQL. Some rules were applied to find the missing information, like repeated hours for the same day are always in the afternoon and the first hour after a new day are in the morning.

### Removing locations that don't have any appointment
Not all locations provide all services, and the ones without any entry were removed from that service. They could also be removed because during the time the data was collected no new appointment was created. No model can be trained without data, so their removal is necessary in either case.

### Dealing with missing data
The data contains certain periods of time missing, either because the website or the scrapping script were down. The script will will find the next available appointment for each entry and calculate how much time it takes until it; if an appointment is available, this distance is zero.
Just before a gap in measurement, the distance will either be zero (appointment available) or it will calculate the distance in hours until the next appointment. This can introduce an outlier depending of the time elapsed without data collection and to avoid it a fake appointment was created as last value before a gap. This also introduces an error, but it is less variable than an unknown gap.

Finally, a fake event is created as the very last entry of each location beacuse the data will later be "stacked" and all columns will be merged. This step is to prevent leaking information between locations, because the next available appointment would be located in the next location.
    
### Stacking by melting
it was decided to stack all columns together and have one column with all the next appointments and one with the location labels instead of one column per 
location to make it easier to apply functions to calculate functions of interest, like distance between next appointments and counts.

### Calculating the next appointment
A countdown of lines was calculated for each value that returns when the next appointmnet is going to take place. If it exists for that time, the difference is zero. This countdown is used to get the next appointmnet and finally the time it will take for the appointment to become available. ***This will be the value to be predicted by the model***. At this point all columns that will not be used were dropped.

### Checking for FFT (tentative)
Fast Fourier Transform was applied for checking if any frequency of appointments occured in the data and if it could be used for predicting the next appointments. The results were inconclusive and the method was aborted.

### Adding categorical values for dates
Finally, the weekdays, hours and minutes were transformed into categorical columns for appling the final model.

## Model construction

### Parameters
Finally, the model was built by using the categorical time columns and predicting when the next appointment would be available. 

Days, hours and minutes were one hot encoded to serve as input. Two main kinds of categorical transformation for days were possible: day of the month and weekday. Weekdays returned better results, which indicates knowing how far in the week one is when trying to schedule an appointment is more usefull than having the day of the month information available.

### Construction
A custom class and factory were built for the model:
- The factory contained the instructions for one hot encoding and a blank model with pre-determined hyperparameters in a form of a pipeline.
- The class received the cleaned data from the earlier steps, split it by location and called the factory. As it dynamically called new blank models, each one was independent from each other, which was used to predict different services and locations.

The idea is to apply the model 

### Hyperparameters
Once the model was ready, a GridSearchCV was run on several sets of hyperparameters and the one with best score was chosen. In order to use the GridSearchCV the dataset was split by location and the one hot encoder transformation was manually applied.

### Evaluation
As a timeseries has time as dependent variable, the dataset was split into the first 80% and the last 20% of observations, to avoid data leakage from the future to present.

The scope of this project is to make one model for all services and locations, hence the evaluation was made on one location at a time.



## Model deployment

### Final choice
After optimizing the model with random forest regressor and ridge, the Root Mean Square Error (RMSE) was calculated for each one. The ridge model had an error as small as the random forest, with the advantage of being faster and easier to explain and faster

In [42]:
w = widgets.Dropdown(
    options=constants.locations, #locations stored in a file called constants
    value='Bakers_Basin',
    description='Number:',
    disabled=False,
)
display(w)

Dropdown(description='Number:', index=1, options=('time_stamp', 'Bakers_Basin', 'Bayonne', 'Camden', 'Cherry_H…

Loads models

In [52]:
w.value

'Camden'

In [86]:
model_name = 'initial_permit_' + w.value + '_model' # 
try:
    model = pickle.load(open(model_name, 'rb'))
    now = datetime.today() #prediction will be based on current time
    df_predict = pd.DataFrame({'days_cat': [now.weekday()], 'hours_cat':[now.hour], 'minutes_cat':[now.minute]}) #now.weekday()
    [results] = model.predict(df_predict) / 3.6e+12 #nano seconds to hours
    predicted_time = now + timedelta(hours = results)
    print('The best time to try to schedule an appointment is: '\
        + predicted_time.strftime('%x, %I:%m %p'))
except FileNotFoundError:
    model = 'no_model'
    print('no model exists for that combination. Does that location provides that service?')

The best time to try to schedule an appointment is: 05/30/22, 02:05 PM


## Conclusion

This model was able to predict a better time for returning to the website and scheduling an appointment online for a given location. The services offered by them are often vital and this tool aims to help people by providing them a better chance to use these services.

The dataset was collected during the period of five months and there is room for improvement; the data is still being collected and can be used to improve the model.

The final model used was a linear one, as it provided an error as low as a more complex models. It has the advantage of having a higher explainability and a smaller training time.

*Cesar Krischer, 2022*